# Coverage of ISIC sectors (high level) by EX sectors and EI activities
based on bridge matrix and ecoinvent ISIC classfications

In [1]:
import pandas as pd
import pickle
from itertools import islice

pd.options.display.float_format = '{:.3g}'.format

# Load data

In [2]:
scenario = '2019-12-02_WW'  # must be the same as the HIOT Excel sheetname
#scenario = 'sensitivity_no_service_sectors'

In [3]:
DATA_PATH = "../data/"
SCENARIO_PATH = "../results/" + scenario + "/"
print(SCENARIO_PATH)

../results/2019-12-02_WW/


In [4]:
def load_pickle(filename):
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
        print(len(data))
        if isinstance(data, dict):
            print(list(islice(data.items(), 2)))
    return data

### File names

In [5]:
# ecoinvent
activity_overview = f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results.xlsx"

# EXIOBASE
hiot_results = f"{DATA_PATH}exiobase/hiot_results_PTM_v2.xlsx"
hiot_results_sheet = "v_3_3_18_2019-11-22"

# Matching
product_matching = f"{DATA_PATH}matching/match_v4.xlsx"
product_matching_sheet = 'concordance'
process_matching = f"{DATA_PATH}matching/match_v2_electricity.xlsx"
region_matching = f"{DATA_PATH}matching/matching_ecoinvent_Exiobase_geographies.xlsx"
region_matching_sheet = "matched_191117"

# ISIC classifications

### ISIC file
This file includes isic classifications level 1-4 plus additionally level 5 classifications from ecoinvent

In [6]:
ISIC = pd.read_excel(f"{DATA_PATH}matching/isic_codes_with_ecoinvent_level5.xlsx")
print(ISIC.shape)
ISIC.head(2)

(783, 18)


,Code,Description,Level 1,Level 2,Level 3,Level 4,Level 5,Top category,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5
0,A,"Agriculture, forestry and fishing",1,0,0,0,0,A,A,NaN,NaN,NaN,NaN,"Agriculture, forestry and fishing",NaN,NaN,NaN,NaN
1,01,"Crop and animal production, hunting and relate...",0,1,0,0,0,A,A,1,NaN,NaN,NaN,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",NaN,NaN,NaN


In [7]:
def get_ISIC_levelX(description, level=1, debug=False):
    """Return a higher or lower level ISIC names and codes, if possible."""
    lookup_columns = ['Description 1', 'Description 2', 'Description 3', 'Description 4', 'Description 5']
    result = None
    
    for col in lookup_columns:
        # print(col)
        if description in ISIC[col].tolist():
            indices = ISIC[col].index[ISIC[col] == description].tolist()
            output_level = 'Description ' + str(level)
            result = set(ISIC.iloc[indices][output_level].tolist())
            if debug:
                print('Found in', col)
                print('Indices:', ISIC[col].index[ISIC[col] == description].tolist())
                print('Level', level, 'match:', result)
       
    return result

In [8]:
d5 = ' Liquid and gaseous fuels from biomass'
d3 = 'Growing of non-perennial crops'
d2 = 'Crop and animal production, hunting and related service activities'
d = 'Agriculture, forestry and fishing'
get_ISIC_levelX(d5, level=2, debug=True)

Found in Description 5
Indices: [162]
Level 2 match: {'Manufacture of coke and refined petroleum products'}


{'Manufacture of coke and refined petroleum products'}

# Results template (ISIC level 1)

In [9]:
DF = ISIC[['Code', 'Description']].loc[ISIC['Level 1'] == 1]
DF.reset_index(inplace=True, drop=True)
DF

,Code,Description
0,A,"Agriculture, forestry and fishing"
1,B,Mining and quarrying
2,C,Manufacturing
3,D,"Electricity, gas, steam and air conditioning s..."
4,E,"Water supply; sewerage, waste management and r..."
5,F,Construction
6,G,Wholesale and retail trade; repair of motor ve...
7,H,Transportation and storage
8,I,Accommodation and food service activities
9,J,Information and communication


# Ecoinvent

### ecoinvent activity overview

In [32]:
AO = pd.read_excel(f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results_with_ISIC_details.xlsx")
#ao.set_index(0, inplace=True)
print(AO.shape)
AO = AO[AO['specialActivityType'] == 'ordinary transforming activity']
print(AO.shape)
AO.head(2)

(14889, 57)
(10805, 57)


,Unnamed: 0,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,...,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,0,"('ecoinvent 3.4 cutoff', '60274947cf82e6633d8c...",2607,2d83c1a0-0c9b-4589-bb9b-6a81e815f5e2,2d83c1a0-0c9b-4589-bb9b-6a81e815f5e2_4c2b1cc3-...,[sulfonyl]urea-compound production,RER,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
1,1,"('ecoinvent 3.4 cutoff', '7dd298333a92144698cf...",5307,59913008-58ba-4f1a-8bf2-86925bbd6005,59913008-58ba-4f1a-8bf2-86925bbd6005_4c2b1cc3-...,[sulfonyl]urea-compound production,RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN


In [34]:
AOnRoW = AO[AO['geography'] != 'RoW']
AOnRoW.shape

(7986, 57)

In [35]:
# number of unique products
print(len(AO.name.unique()))
print(len(AOnRoW.name.unique()))

2851
2851


In [37]:
# number of unique product-activity combinations (what we describe in the paper as "products")
print(len((AO.name+"//"+AO.activityName).unique()))
print(len((AOnRoW.name+AOnRoW.activityName).unique()))

4208
4150


In [50]:
#These 58 activities have ONLY a RoW representation (didn't know that this exists in ecoinvent...)
set((AO.name+"//"+AO.activityName).unique()) - set((AOnRoW.name+"//"+AOnRoW.activityName).unique())

{'electricity, for reuse in municipal waste incineration only//treatment of liquid crystal display, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of raw sewage sludge, municipal incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, IT accessory, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, cathode ray tube display, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, desktop computer, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, industrial device, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical trea

In [38]:
# number of unique product-activity-geography combinations
print(len((AO.name+AO.activityName+AO.geography).unique()))
print(len((AOnRoW.name+AOnRoW.activityName+AOnRoW.geography).unique()))

10803
7984


In [39]:
diff = pd.concat([AO.groupby, AOnRoW]).drop_duplicates(keep=False)
diff

,Unnamed: 0,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,...,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
1,1,"('ecoinvent 3.4 cutoff', '7dd298333a92144698cf...",5307,59913008-58ba-4f1a-8bf2-86925bbd6005,59913008-58ba-4f1a-8bf2-86925bbd6005_4c2b1cc3-...,[sulfonyl]urea-compound production,RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
3,3,"('ecoinvent 3.4 cutoff', '35e16ca3f2007a0186f0...",7754,8419276e-31ac-4859-b9ce-05d4f05199c6,8419276e-31ac-4859-b9ce-05d4f05199c6_1be4f7e4-...,[thio]carbamate-compound production,RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
5,5,"('ecoinvent 3.4 cutoff', '555a1f49dfd1dd13e5a2...",5781,61ee25d4-accb-4d44-b381-9b78e0f057d5,61ee25d4-accb-4d44-b381-9b78e0f057d5_2487d46f-...,"acetamide-anillide-compound production, unspec...",RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
7,7,"('ecoinvent 3.4 cutoff', '49aa7ecf75011a9cdbac...",12231,d1b0e57c-90c3-49c1-b6e3-19e5e162a569,d1b0e57c-90c3-49c1-b6e3-19e5e162a569_906ace6c-...,aclonifen production,RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
9,9,"('ecoinvent 3.4 cutoff', '14406fd5b1ec4fbe5112...",8992,99d608c8-6d0e-4ee6-96aa-783e2a30690f,99d608c8-6d0e-4ee6-96aa-783e2a30690f_bc06c7e3-...,atrazine production,RoW,2000-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of chemicals and chemical products,Manufacture of other chemical products,Manufacture of pesticides and other agrochemic...,NaN,C:Manufacturing,20:Manufacture of chemicals and chemical products,202:Manufacture of other chemical products,2021:Manufacture of pesticides and other agroc...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14861,14861,"('ecoinvent 3.4 cutoff', '06c94c3c4da6e53b4873...",14264,f52990d1-7b30-4be6-8af7-719934c01f70,f52990d1-7b30-4be6-8af7-719934c01f70_a90c974c-...,"tube insulation production, elastomere",RoW,1997-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of rubber and plastics products,Manufacture of rubber products,Manufacture of other rubber products,NaN,C:Manufacturing,22:Manufacture of rubber and plastics products,221:Manufacture of rubber products,2219:Manufacture of other rubber products,NaN
14863,14863,"('ecoinvent 3.4 cutoff', '861c0a3ea73ad63e000d...",2288,27f369f3-985d-4146-a3e3-07a008c7a38c,27f369f3-985d-4146-a3e3-07a008c7a38c_dd0af955-...,uranium hexafluoride production,RoW,1982-01-01,2017-12-31,ordinary transforming activity,...,Manufacturing,Manufacture of basic metals,Manufacture of basic precious and other non-fe...,Manufacture of basic precious and other non

In [11]:
EI_ISIC = AO['key'].groupby(AO['Description 1']).count()
EI_ISIC.name = 'ecoinvent, w/o markets'
#EI_ISIC = EI_ISIC.astype(int)
EI_ISIC

Description 1
Administrative and support service activities                             85
Agriculture, forestry and fishing                                        808
Construction                                                             525
Electricity, gas, steam and air conditioning supply                     3162
Information and communication                                             13
Manufacturing                                                           3907
Mining and quarrying                                                     284
Other service activities                                                   2
Professional, scientific and technical activities                          4
Real estate activities                                                   200
Transportation and storage                                               248
Water supply; sewerage, waste management and remediation activities     1504
Wholesale and retail trade; repair of motor vehicles and motor

In [12]:
print('Activities in ecoinvent that have an ISIC classification:', EI_ISIC.sum())
print('Activities that lack and ISIC classification:', AO.shape[0] - EI_ISIC.sum())
# these are "electricity ... production mixes" and "Recycled Content cut-off" activities

Activities in ecoinvent that have an ISIC classification: 10760
Activities that lack and ISIC classification: 45


In [13]:
DF_EI = DF.join(EI_ISIC, 'Description').fillna(value=0).astype({'ecoinvent, w/o markets': int})
DF_EI

,Code,Description,"ecoinvent, w/o markets"
0,A,"Agriculture, forestry and fishing",808
1,B,Mining and quarrying,284
2,C,Manufacturing,3907
3,D,"Electricity, gas, steam and air conditioning s...",3162
4,E,"Water supply; sewerage, waste management and r...",1504
5,F,Construction,525
6,G,Wholesale and retail trade; repair of motor ve...,18
7,H,Transportation and storage,248
8,I,Accommodation and food service activities,0
9,J,Information and communication,13


### Ecoinvent "reduced"

1. only "ordinary transforming activities", i.e. no markets, production mixes, market groups
2. only unique product-activity combinations, i.e. ignoring p-a combinations that occur in several geographies

#### Group by product and activity to "drop" the geographies

In [14]:
AOng = AO.groupby(['name', 'activityName', 'Description 1']).count()
#AOng = AOt.groupby(['name', 'activityName']).count()
#AOng = AOt.groupby(['activityName', 'name',]).count()
AOng.reset_index(inplace=True)  
print(AOng.shape)
AOng

(4164, 57)


,name,activityName,Description 1,Unnamed: 0,key,#,id,filename,geography,startDate,endDate,specialActivityType,technologyLevel,inheritance status,parentActivityId,tags,synonyms,ISIC v4,ISIC code,ISIC name,specialty production,constrained market,accessRestrictedTo,dataEntryBy,dataGenerator,isActiveAuthor,group,CPC,amount,unitName,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"('IPCC 2013', 'climate change', 'GWP 100a')",Unnamed: 0.1,#activities,Code (ecoinvent),Description (ecoinvent),ISIC level,Code 1,Code 2,Code 3,Code 4,Code 5,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
1,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,1,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
2,1-butanol,hydroformylation of propylene,Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
3,1-pentanol,hydroformylation of butene,Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
4,1-propanol,1-propanol production,Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
5,"2,3-dimethylbutan",molecular sieve separation of naphtha,Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,1,0,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
6,"2,4-di-tert-butylphenol","2,4-di-tert-butylphenol production",Manufacturing,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0
7,"2,4-dichlorophenol","2,4-dichlorophenol production",Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
8,"2,4-dichlorotoluene","2,4-dichlorotoluene production",Manufacturing,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,0,2,2,2,2,0
9,"2,4-dinitrotoluene","2,4-dinitrotoluene production",Manufacturing,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0


In [15]:
EIng = AOng['Description 1'].groupby(AOng['Description 1']).count()
EIng.name = 'ecoinvent, w/o markets and geographies'
print(EIng.sum())
EIng

4164


Description 1
Administrative and support service activities                             26
Agriculture, forestry and fishing                                        398
Construction                                                             279
Electricity, gas, steam and air conditioning supply                      343
Information and communication                                              6
Manufacturing                                                           2059
Mining and quarrying                                                     135
Other service activities                                                   1
Professional, scientific and technical activities                          2
Real estate activities                                                    13
Transportation and storage                                               151
Water supply; sewerage, waste management and remediation activities      741
Wholesale and retail trade; repair of motor vehicles and motor

#### Join with results DF

In [16]:
DF_EIr = DF_EI.join(EIng, 'Description').fillna(value=0).astype({'ecoinvent, w/o markets and geographies': int})
DF_EIr.to_clipboard()
DF_EIr

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies"
0,A,"Agriculture, forestry and fishing",808,398
1,B,Mining and quarrying,284,135
2,C,Manufacturing,3907,2059
3,D,"Electricity, gas, steam and air conditioning s...",3162,343
4,E,"Water supply; sewerage, waste management and r...",1504,741
5,F,Construction,525,279
6,G,Wholesale and retail trade; repair of motor ve...,18,10
7,H,Transportation and storage,248,151
8,I,Accommodation and food service activities,0,0
9,J,Information and communication,13,6


# EXIOBASE

### Load manually matched HIOT sectors to ISIC Code1 

In [17]:
HIOT_ISIC = pd.read_excel(f"{DATA_PATH}matching/ISIC_level1_to_EXIOBASE.xlsx")
HIOT_ISIC.rename(columns={"ISIC Code 1": "Code"}, inplace=True)
HIOT_ISIC.head()

,#,Code,ProductTypeName_of_hiot,product code 1,product code 2
0,1,A,Cultivation of paddy rice,c01.a,P_PARI
1,2,A,Cultivation of wheat,c01.b,P_WHEA
2,3,A,Cultivation of cereal grains nec,c01.c,P_OCER
3,4,A,"Cultivation of vegetables, fruit, nuts",c01.d,P_FVEG
4,5,A,Cultivation of oil seeds,c01.e,P_OILS


In [18]:
HIOT_nonregionalized = HIOT_ISIC.groupby('Code')['#'].nunique()
HIOT_nonregionalized

Code
A    19
B    15
C    48
D    17
E    35
F     1
G     4
H     7
I     1
J     1
K     3
L     2
M     2
N     1
O     1
P     1
Q     1
R     1
S     2
T     1
U     1
Name: #, dtype: int64

### Instead of multiplying by 48, get actual numbers of sectors (not always fully regionalized) from "hiot_results.xlsx"

In [19]:
HIOT = pd.read_excel(hiot_results, sheet_name=hiot_results_sheet)
print(HIOT.shape)
HIOT

(8200, 10)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit
0,1,AU,Cultivation of paddy rice,c01.a,P_PARI,tonnes,1.88,kg,1.88,kg CO2-eq/kg
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.361,kg,0.361,kg CO2-eq/kg
2,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.313,kg,0.313,kg CO2-eq/kg
3,4,AU,"Cultivation of vegetables, fruit, nuts",c01.d,P_FVEG,tonnes,1.98,kg,1.98,kg CO2-eq/kg
4,5,AU,Cultivation of oil seeds,c01.e,P_OILS,tonnes,0.43,kg,0.43,kg CO2-eq/kg
5,6,AU,"Cultivation of sugar cane, sugar beet",c01.f,P_SUGB,tonnes,0.124,kg,0.124,kg CO2-eq/kg
6,7,AU,Cultivation of plant-based fibers,c01.g,P_FIBR,tonnes,1.58,kg,1.58,kg CO2-eq/kg
7,8,AU,Cultivation of crops nec,c01.h,P_OTCR,tonnes,71.9,kg,71.9,kg CO2-eq/kg
8,9,AU,Cattle farming,c01.i,P_CATL,tonnes,66.4,kg,66.4,kg CO2-eq/kg
9,10,AU,Pigs farming,c01.j,P_PIGS,tonnes,4.92,kg,4.92,kg CO2-eq/kg


#### Exclude GLO, EU-28 and rows where GWP=0 (products not produced in a country)

In [20]:
HIOTr = HIOT[HIOT['CountryCode'] != 'GLO']
HIOTr = HIOTr[HIOTr['CountryCode'] != 'EU-28']
print(HIOTr.shape)
HIOTr = HIOTr[HIOTr['CarbonFootprint'] != 0]
print(HIOTr.shape)

(7872, 10)
(6692, 10)


#### Merge with ISIC code 1

In [21]:
HIOTm = pd.merge(HIOTr, HIOT_ISIC[['ProductTypeName_of_hiot','Code']], on=["ProductTypeName_of_hiot"], how="inner")
print(HIOTm.shape)
HIOTm.head()

(6692, 11)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,Code
0,1,AU,Cultivation of paddy rice,c01.a,P_PARI,tonnes,1.88,kg,1.88,kg CO2-eq/kg,A
1,493,BR,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.82,kg,0.82,kg CO2-eq/kg,A
2,657,BG,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.506,kg,0.506,kg CO2-eq/kg,A
3,985,CN,Cultivation of paddy rice,c01.a,P_PARI,tonnes,2.38,kg,2.38,kg CO2-eq/kg,A
4,1969,FR,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.365,kg,0.365,kg CO2-eq/kg,A


In [22]:
HIOT_regionalized = HIOTm.groupby('Code')['#'].nunique()
HIOT_regionalized

Code
A     761
B     437
C    2185
D     572
E    1347
F      48
G     192
H     336
I      48
J      48
K     144
L      96
M      96
N      48
O      48
P      48
Q      48
R      48
S      96
T      46
Name: #, dtype: int64

## Merge ecoinvent and EXIOBASE data

In [23]:
HIOT_nonregionalized

Code
A    19
B    15
C    48
D    17
E    35
F     1
G     4
H     7
I     1
J     1
K     3
L     2
M     2
N     1
O     1
P     1
Q     1
R     1
S     2
T     1
U     1
Name: #, dtype: int64

In [24]:
DF = pd.merge(DF_EIr, HIOT_nonregionalized, on=["Code"], how="outer").fillna(value=0)
DF.rename(columns={"#": "EXIOBASE, non-regionalized"}, inplace=True)
DF = pd.merge(DF, HIOT_regionalized, on=["Code"], how="outer").fillna(value=0)
DF.rename(columns={"#": "EXIOBASE"}, inplace=True)
DF

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized",EXIOBASE
0,A,"Agriculture, forestry and fishing",808,398,19,761
1,B,Mining and quarrying,284,135,15,437
2,C,Manufacturing,3907,2059,48,2.18e+03
3,D,"Electricity, gas, steam and air conditioning s...",3162,343,17,572
4,E,"Water supply; sewerage, waste management and r...",1504,741,35,1.35e+03
5,F,Construction,525,279,1,48
6,G,Wholesale and retail trade; repair of motor ve...,18,10,4,192
7,H,Transportation and storage,248,151,7,336
8,I,Accommodation and food service activities,0,0,1,48
9,J,Information and communication,13,6,1,48


# Present data

#### Ratios

In [25]:
DF['Ratio, non-regionalized'] = DF['ecoinvent, w/o markets and geographies']/DF['EXIOBASE, non-regionalized']
DF['Ratio'] = DF['ecoinvent, w/o markets']/DF['EXIOBASE']
DF

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized",EXIOBASE,"Ratio, non-regionalized",Ratio
0,A,"Agriculture, forestry and fishing",808,398,19,761,20.9,1.06
1,B,Mining and quarrying,284,135,15,437,9,0.65
2,C,Manufacturing,3907,2059,48,2.18e+03,42.9,1.79
3,D,"Electricity, gas, steam and air conditioning s...",3162,343,17,572,20.2,5.53
4,E,"Water supply; sewerage, waste management and r...",1504,741,35,1.35e+03,21.2,1.12
5,F,Construction,525,279,1,48,279,10.9
6,G,Wholesale and retail trade; repair of motor ve...,18,10,4,192,2.5,0.0938
7,H,Transportation and storage,248,151,7,336,21.6,0.738
8,I,Accommodation and food service activities,0,0,1,48,0,0
9,J,Information and communication,13,6,1,48,6,0.271


#### Column naming

In [26]:
column_names = [
"Code",
"Description",
"ecoinvent, w/o markets and geographies",
"EXIOBASE, non-regionalized",
"Ratio, non-regionalized",
"ecoinvent, w/o markets",
"EXIOBASE",
"Ratio",
]

In [27]:
DF = DF[column_names]
DF

,Code,Description,"ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized","Ratio, non-regionalized","ecoinvent, w/o markets",EXIOBASE,Ratio
0,A,"Agriculture, forestry and fishing",398,19,20.9,808,761,1.06
1,B,Mining and quarrying,135,15,9,284,437,0.65
2,C,Manufacturing,2059,48,42.9,3907,2.18e+03,1.79
3,D,"Electricity, gas, steam and air conditioning s...",343,17,20.2,3162,572,5.53
4,E,"Water supply; sewerage, waste management and r...",741,35,21.2,1504,1.35e+03,1.12
5,F,Construction,279,1,279,525,48,10.9
6,G,Wholesale and retail trade; repair of motor ve...,10,4,2.5,18,192,0.0938
7,H,Transportation and storage,151,7,21.6,248,336,0.738
8,I,Accommodation and food service activities,0,1,0,0,48,0
9,J,Information and communication,6,1,6,13,48,0.271


#### Totals

In [28]:
DF_FINAL = DF.append(DF.sum(numeric_only=True), ignore_index=True)
DF_FINAL.to_clipboard()
DF_FINAL

,Code,Description,"ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized","Ratio, non-regionalized","ecoinvent, w/o markets",EXIOBASE,Ratio
0,A,"Agriculture, forestry and fishing",398,19,20.9,808,761,1.06
1,B,Mining and quarrying,135,15,9,284,437,0.65
2,C,Manufacturing,2.06e+03,48,42.9,3.91e+03,2.18e+03,1.79
3,D,"Electricity, gas, steam and air conditioning s...",343,17,20.2,3.16e+03,572,5.53
4,E,"Water supply; sewerage, waste management and r...",741,35,21.2,1.5e+03,1.35e+03,1.12
5,F,Construction,279,1,279,525,48,10.9
6,G,Wholesale and retail trade; repair of motor ve...,10,4,2.5,18,192,0.0938
7,H,Transportation and storage,151,7,21.6,248,336,0.738
8,I,Accommodation and food service activities,0,1,0,0,48,0
9,J,Information and communication,6,1,6,13,48,0.271
